In [1]:
import torch, platform
print("Python:", platform.python_version())
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
# macOS Metal (MPS) fallback
if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    print("MPS available: True")


Python: 3.11.9
PyTorch: 2.7.0.dev20250309+cu128
CUDA available: True
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [6]:
from transformers import pipeline

sent = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
print(sent("Hugging Face makes NLP easy!"))

zshot = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli"  # compact MNLI model
)
print(zshot(
    "I loved the buttery croissant but the coffee was burnt.",
    candidate_labels=["food quality", "service", "price", "ambience"]
))


Device set to use cuda:0
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


[{'label': 'POSITIVE', 'score': 0.9987070560455322}]


Device set to use cuda:0


{'sequence': 'I loved the buttery croissant but the coffee was burnt.', 'labels': ['ambience', 'food quality', 'price', 'service'], 'scores': [0.9803847074508667, 0.008877531625330448, 0.005549285560846329, 0.00518846744671464]}


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

ckpt = "distilbert-base-uncased-finetuned-sst-2-english"  # note: includes a trained classifier head
tok = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSequenceClassification.from_pretrained(ckpt)

inputs = tok("This course will be awesome.", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
probs = logits.softmax(-1).squeeze()
print("Label probs:", {model.config.id2label[i]: float(probs[i]) for i in range(len(probs))})


Label probs: {'NEGATIVE': 0.00013041975034866482, 'POSITIVE': 0.9998695850372314}


In [7]:
# Works on recent huggingface_hub versions
from huggingface_hub import scan_cache_dir
info = scan_cache_dir()
print(info)                      # human-readable summary (includes cache path)
print("Repos cached:", len(info.repos))



HFCacheInfo(size_on_disk=2062822827, repos=frozenset({CachedRepoInfo(repo_id='glue', repo_type='dataset', repo_path=WindowsPath('C:/Users/fiona/.cache/huggingface/hub/datasets--glue'), size_on_disk=1068696, nb_files=4, revisions=frozenset({CachedRevisionInfo(commit_hash='bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c', snapshot_path=WindowsPath('C:/Users/fiona/.cache/huggingface/hub/datasets--glue/snapshots/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c'), size_on_disk=1068696, files=frozenset({CachedFileInfo(file_name='README.md', file_path=WindowsPath('C:/Users/fiona/.cache/huggingface/hub/datasets--glue/snapshots/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/README.md'), blob_path=WindowsPath('C:/Users/fiona/.cache/huggingface/hub/datasets--glue/snapshots/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/README.md'), size_on_disk=35296, blob_last_accessed=1746450457.4062219, blob_last_modified=1746445224.71849), CachedFileInfo(file_name='test-00000-of-00001.parquet', file_path=WindowsPath('C:/Users/fiona/.ca